In [ ]:
!pip install --break-system-packages git+https://github.com/openai/whisper.git

In [ ]:
!pip install --break-system-packages pyannote.audio torchaudio # pydub not strictly needed if only ffmpeg is used for audio proc

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.1/821.1 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import whisper
from pyannote.audio import Pipeline
import torch
import re
import os
import subprocess

In [ ]:
input_audio_path = "/content/001_t2.wav"
clean_audio_path = "cleaned_audio_for_asr_and_diarization.wav"

In [ ]:
HUGGING_FACE_ACCESS_TOKEN = "hf_" # <-- REPLACE THIS!

In [ ]:
# --- Step 1: Enhanced Audio Preprocessing with ffmpeg and Duration Verification ---
print("--- Starting Audio Preprocessing ---")

# First, get the duration of the original input file for comparison
print(f"--- Verifying Original Input Audio Duration ({input_audio_path}) ---")
ffprobe_command_input = ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", input_audio_path]
try:
    input_duration_output = subprocess.run(ffprobe_command_input, check=True, capture_output=True, text=True).stdout.strip()
    original_input_duration_seconds = float(input_duration_output)
    print(f"Original input audio duration: {original_input_duration_seconds:.2f} seconds")
except Exception as e:
    print(f"Error getting original input audio duration: {e}. Please ensure the input WAV file exists and is valid.")
    # Exit or raise error if input audio duration cannot be determined
    raise

# FFmpeg command using the original successful filters
ffmpeg_command = [
    "ffmpeg",
    "-i", input_audio_path,
    "-acodec", "pcm_s16le",
    "-ac", "1",
    "-ar", "16000",
    # Reverted to the original working filters: loudnorm, highpass, lowpass
    "-af", "loudnorm=I=-16:TP=-1.5:LRA=11, highpass=f=200, lowpass=f=3000",
    "-y", clean_audio_path # -y to overwrite output file without asking
]

try:
    result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
    if result.stdout:
        print("FFmpeg stdout:")
        print(result.stdout)
    if result.stderr:
        print("FFmpeg stderr (might contain warnings/info):")
        print(result.stderr)
    print(f"--- Audio Preprocessing Complete. Cleaned audio saved to {clean_audio_path} ---")

    # Verify duration of the cleaned audio file
    print(f"--- Verifying Cleaned Audio Duration ({clean_audio_path}) ---")
    ffprobe_command_output = ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", clean_audio_path]
    duration_output = subprocess.run(ffprobe_command_output, check=True, capture_output=True, text=True).stdout.strip()
    try:
        cleaned_audio_duration_seconds = float(duration_output)
        print(f"Cleaned audio duration: {cleaned_audio_duration_seconds:.2f} seconds")
        if abs(original_input_duration_seconds - cleaned_audio_duration_seconds) > 0.1: # Allow for small floating point differences
            print(f"WARNING: Cleaned audio duration ({cleaned_audio_duration_seconds:.2f}s) significantly differs from input ({original_input_duration_seconds:.2f}s). This might indicate a truncation problem during FFmpeg processing.")
        else:
            print("Cleaned audio duration matches input audio duration.")
    except ValueError:
        print(f"Could not parse duration from ffprobe for cleaned audio: {duration_output}")

except subprocess.CalledProcessError as e:
    print(f"--- FFmpeg Error ---")
    print(f"Command: {' '.join(e.cmd)}")
    print(f"Return Code: {e.returncode}")
    print(f"STDOUT:\n{e.stdout}")
    print(f"STDERR:\n{e.stderr}")
    print(f"--- Audio Preprocessing Failed. Cannot proceed. ---")
    raise e
except FileNotFoundError:
    print("--- FFmpeg/FFprobe not found ---")
    print("Please ensure FFmpeg and FFprobe are installed and accessible in your environment's PATH.")
    raise

--- Starting Audio Preprocessing ---
--- Verifying Original Input Audio Duration (/content/001_t2.wav) ---
Original input audio duration: 238.30 seconds
FFmpeg stderr (might contain warnings/info):
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enab

In [ ]:
# --- Step 2: Whisper Transcription ---
print("--- Starting Whisper Transcription ---")

--- Starting Whisper Transcription ---


In [ ]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [01:31<00:00, 33.6MiB/s]


In [ ]:
initial_prompt_text = (
    "Axis Maxlife Insurance, Policy number, fund value, risk term, Premium Due, Due date, Sum Assured, Policy Status, "
    "Late Fee, Google pay, gpay, phone pay, paytm, netbacking"
)

In [ ]:
whisper_result = model.transcribe(
    clean_audio_path,
    language="ta",       # Explicitly set source language as Tamil
    task="translate",     # Translate from Tamil to English
    verbose=True,
    initial_prompt=initial_prompt_text
)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:07.000]  Please add any key to the search option.
[00:07.000 --> 00:14.000]  Hello Sir, My name is Arookya Mehri. Can I talk to Mr. Subbaraj Sir?
[00:14.000 --> 00:17.000]  Yes, I am coming.
[00:17.000 --> 00:23.000]  I have called from Maxlife Insurance. Can I talk about the policy? Are you free?
[00:23.000 --> 00:25.000]  Yes, I am free.
[00:25.000 --> 00:33.000]  Thank you. You have taken a policy from your name. Maxlife Fast Strike Super Plan, Policy number is 00603260.
[00:33.000 --> 00:44.000]  The due date is 30th September 2021. The amount to be paid is 1,20,000. The policy status is discontinued. Can I know why you are not paying?
[00:44.000 --> 00:47.000]  Finance problem.
[00:47.000 --> 00:48.000]  Sorry Sir.
[00:48.000 --> 00:52.000]  Finance problem. I am asking for 1,20,000.
[00:52.000 --> 00:54.000]  Yes Sir.
[00:54.000 --> 00:55.000]  Thank you.
[00:55.000 --> 00:57.000]  Can I take the money from 1,20,000?
[00:57.000 --> 00:58.000]  Yes Sir.
[00:58.00

In [ ]:
print("--- Whisper Transcription Complete ---")

--- Whisper Transcription Complete ---


In [ ]:
# --- Step 3: Speaker Diarization with pyannote.audio ---
print("\n--- Starting Speaker Diarization ---")
try:
    pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization-3.1",
        use_auth_token=HUGGING_FACE_ACCESS_TOKEN
    )

    # Send pipeline to GPU (if available)
    if torch.cuda.is_available():
        pipeline.to(torch.device("cuda"))
        print("Pyannote.audio moved to GPU.")
    else:
        print("CUDA not available, running pyannote.audio on CPU. This might be slow.")

    diarization = pipeline(clean_audio_path)
    print("--- Speaker Diarization Complete ---")

except Exception as e:
    print(f"--- Speaker Diarization Failed ---")
    print(f"Error: {e}")
    print("Please ensure your Hugging Face Access Token is correct and has access to pyannote/speaker-diarization-3.1.")
    # If diarization fails, we can still proceed with transcription but without speaker labels
    diarization = None # Set diarization to None if it failed


--- Starting Speaker Diarization ---


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

CUDA not available, running pyannote.audio on CPU. This might be slow.


/usr/local/lib/python3.11/dist-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


--- Speaker Diarization Complete ---


In [ ]:
# --- Step 4: Combine and Format Output ---
print("\n--- Generating Dialogue Output ---")
dialogue_output = []

# Helper function to find the dominant speaker for a given time segment
def get_dominant_speaker_for_segment(start_time, end_time, diarization_result):
    if not diarization_result: # If diarization failed
        return "Unknown"

    speakers_in_segment = {}
    for segment, _, speaker_label in diarization_result.itertracks(yield_label=True):
        # Calculate overlap between transcription segment and diarization segment
        overlap_start = max(start_time, segment.start)
        overlap_end = min(end_time, segment.end)
        overlap_duration = max(0, overlap_end - overlap_start)

        if overlap_duration > 0:
            speakers_in_segment[speaker_label] = speakers_in_segment.get(speaker_label, 0) + overlap_duration

    if not speakers_in_segment:
        return "Unknown"

    # Return the speaker with the most overlap
    return max(speakers_in_segment, key=speakers_in_segment.get)

# Group segments by speaker for better dialogue flow (experimental, can be adjusted)
current_speaker = None
current_text = []
current_start = -1
current_end = -1

for i, segment in enumerate(whisper_result["segments"]):
    start = segment['start']
    end = segment['end']
    text = segment['text'].strip()

    speaker = get_dominant_speaker_for_segment(start, end, diarization)

    if speaker == current_speaker and current_speaker is not None and (start - current_end) < 2.0: # Merge if same speaker and short pause
        current_text.append(text)
        current_end = end
    else:
        if current_speaker is not None:
            dialogue_output.append(f"Speaker {current_speaker}: {' '.join(current_text)}")
        current_speaker = speaker
        current_text = [text]
        current_start = start
        current_end = end

# Add the last accumulated segment
if current_speaker is not None:
    dialogue_output.append(f"Speaker {current_speaker}: {' '.join(current_text)}")



--- Generating Dialogue Output ---


In [ ]:
# --- Step 5: Print the final dialogue ---
for line in dialogue_output:
    print(line)

print("\n--- Processing Complete ---")

Speaker SPEAKER_00: Please add any key to the search option. Hello Sir, My name is Arookya Mehri. Can I talk to Mr. Subbaraj Sir?
Speaker SPEAKER_01: Yes, I am coming.
Speaker SPEAKER_00: I have called from Maxlife Insurance. Can I talk about the policy? Are you free?
Speaker SPEAKER_01: Yes, I am free.
Speaker SPEAKER_00: Thank you. You have taken a policy from your name. Maxlife Fast Strike Super Plan, Policy number is 00603260. The due date is 30th September 2021. The amount to be paid is 1,20,000. The policy status is discontinued. Can I know why you are not paying?
Speaker SPEAKER_01: Finance problem.
Speaker SPEAKER_00: Sorry Sir.
Speaker SPEAKER_01: Finance problem. I am asking for 1,20,000.
Speaker SPEAKER_00: Yes Sir.
Speaker SPEAKER_01: Thank you. Can I take the money from 1,20,000? Yes Sir.
Speaker SPEAKER_00: So, you have a 5 year locking period. You have taken a policy in 2018. You have paid the due for 18, 19, 20 years. Can you take the amount after paying the due? How mu

In [5]:
# dialogue_output

In [ ]:
# Assuming 'dialogue_output' is the list generated from the previous code execution
# For demonstration purposes, let's use a snippet of your provided dialogue_output
# In your actual notebook, you would use the 'dialogue_output' variable directly
dialogue_output = [
    "Speaker SPEAKER_00: Please add any key to the search option. Hello Sir, My name is Arookya Mehri. Can I talk to Mr. Subbaraj Sir?",
    "Speaker SPEAKER_01: Yes, I am coming.",
    "Speaker SPEAKER_00: I have called from Maxlife Insurance. Can I talk about the policy? Are you free?",
    "Speaker SPEAKER_01: Yes, I am free.",
    "Speaker SPEAKER_00: Thank you. You have taken a policy from your name. Maxlife Fast Strike Super Plan, Policy number is 00603260. The due date is 30th September 2021. The amount to be paid is 1,20,000. The policy status is discontinued. Can I know why you are not paying?",
    "Speaker SPEAKER_01: Finance problem.",
    "Speaker SPEAKER_00: Sorry Sir.",
    "Speaker SPEAKER_01: Finance problem. I am asking for 1,20,000.",
    "Speaker SPEAKER_00: Yes Sir.",
    "Speaker SPEAKER_01: Thank you. Can I take the money from 1,20,000? Yes Sir.",
    "Speaker SPEAKER_00: So, you have a 5 year locking period. You have taken a policy in 2018. You have paid the due for 18, 19, 20 years. Can you take the amount after paying the due? How much? You can take the amount after paying the due.",
    "Speaker SPEAKER_01: How much?",
    "Speaker SPEAKER_00: Sir, you can take the amount according to your fund value. Or you can withdraw the money partially.",
    "Speaker SPEAKER_01: No, I don't have any fund value. I will withdraw it partially.",
    "Speaker SPEAKER_00: Sir, as of now, the amount is 1,87,551. But, if you pay the amount now, your fund will participate in the share market. So, there is a chance that you will get high returns.",
    "Speaker SPEAKER_01: Sir, it has been 3 years and 1,87,551. It is fair in any way.",
    "Speaker SPEAKER_00: Sir, your policy status is discontinued. Your funds are going in discontinued funds. Your fund will participate only if your policy status is active. Sir, you have not paid from 2021. So, how will your fund participate?",
    "Speaker SPEAKER_01: Sir, I have paid the loan of 1,20,000 and paid it. When will the policy end? 5 years.",
    "Speaker SPEAKER_00: You have to pay the loan for 5 years. But, the risk is for 10 years. Your fund will participate only for 10 years. If you need any emergency amount, you can withdraw the money partially. Or you can take the amount. Sir, you can take the amount until the locking period is completed.",
    "Speaker SPEAKER_01: Sir, when did the locking period start?",
    "Speaker SPEAKER_00: You have taken the amount in 2018. In this month? Yes. You have taken the amount in 30th and 31st of the 9th month.",
    "Speaker SPEAKER_01: Can I take the amount in 30th and 31st of the 9th month?",
    "Speaker SPEAKER_00: You can take the amount as soon as the payment is completed.",
    "Speaker SPEAKER_01: Can I take the amount in 20th and 31st of the 9th month?",
    "Speaker SPEAKER_00: Yes. Your locking period is completed with 2022. But, you can take the amount if you pay the loan.",
    "Speaker SPEAKER_01: Where can I get the amount if the payment is completed?",
    "Speaker SPEAKER_00: You can pay the amount in the branch office. Or you can pay the amount online. You can pay the amount online.",
    "Speaker SPEAKER_01: I don't know. You can pay the amount in the branch office. I don't know.",
    "Speaker SPEAKER_00: Sir, I will guide you. You will pay the amount online. I will help you if you get any errors.",
    "Speaker SPEAKER_01: I don't know. I don't know how to pay online. I don't know how to pay online. I don't know how to pay online. You can measure the amount in your dashboard and make the payment close. I don't know how to pay online. But, I will borrow a loan from someone.",
    "Speaker SPEAKER_00: I don't know how to pay online. I don't know how to pay online. Do you know that, this is the post card of your principal? I do not know.",
    "Speaker SPEAKER_01: He is receiving credit withains of cash. In this case, that is a credit card. It is creditor's credit. You can claim it with your credit cards. Now take the payment. Second question. First question. Yes, we can take the amount after the locking period is completed."
]

# --- 1. Define Helper Functions ---

def get_agent_lines(transcript_list):
    """Extracts all lines spoken by Speaker_00 (Agent)."""
    return [line.replace("Speaker SPEAKER_00: ", "") for line in transcript_list if line.startswith("Speaker SPEAKER_00:")]

def get_customer_lines(transcript_list):
    """Extracts all lines spoken by Speaker_01 (Customer)."""
    return [line.replace("Speaker SPEAKER_01: ", "") for line in transcript_list if line.startswith("Speaker SPEAKER_01:")]

# --- 2. Analyze Agent's Greeting ---
print("--- Analyzing Agent's Greeting ---")
agent_first_line = get_agent_lines(dialogue_output)[0].lower() if get_agent_lines(dialogue_output) else ""
greeting_phrases = ["hello sir", "good morning", "good afternoon", "good evening", "namaste sir", "greetings"]
agent_greeted_well = False
for phrase in greeting_phrases:
    if phrase in agent_first_line:
        agent_greeted_well = True
        break

if agent_greeted_well:
    print(f"✅ Agent's Greeting: The agent used a polite greeting: '{agent_first_line}'")
else:
    print(f"❌ Agent's Greeting: The agent's initial greeting was not clearly identified as polite. First line: '{agent_first_line}'")

# --- 3. Analyze Inappropriate Words by Agent ---
print("\n--- Analyzing Inappropriate Words by Agent ---")
all_agent_text = " ".join(get_agent_lines(dialogue_output)).lower()
# You can expand this list of inappropriate words as needed
inappropriate_words = ["damn", "hell", "stupid", "idiot", "nonsense", "rubbish"] # Add more as required
found_inappropriate_words = [word for word in inappropriate_words if word in all_agent_text]

if found_inappropriate_words:
    print(f"❌ Inappropriate Words: The agent used potentially inappropriate words: {', '.join(found_inappropriate_words)}")
else:
    print("✅ Inappropriate Words: No obvious inappropriate words detected from the agent.")

# --- 4. Analyze Agent's Call Closing ---
print("\n--- Analyzing Agent's Call Closing ---")
agent_last_line = ""
# Iterate from the end to find the last agent utterance
for i in range(len(dialogue_output) - 1, -1, -1):
    if dialogue_output[i].startswith("Speaker SPEAKER_00:"):
        agent_last_line = dialogue_output[i].replace("Speaker SPEAKER_00: ", "").lower()
        break

closing_phrases = ["thank you for your time", "have a good day", "goodbye", "bye", "is there anything else", "anything else i can help with"]
agent_closed_well = False
for phrase in closing_phrases:
    if phrase in agent_last_line:
        agent_closed_well = True
        break

if agent_closed_well:
    print(f"✅ Agent's Closing: The agent appears to have closed the call on a good note. Last line: '{agent_last_line}'")
else:
    print(f"❌ Agent's Closing: The agent's closing remark was not clearly identified as a good closing. Last line: '{agent_last_line}'")
    print("   (Note: The transcript provided ends abruptly, so the actual closing might be missing or informal.)")

--- Analyzing Agent's Greeting ---
✅ Agent's Greeting: The agent used a polite greeting: 'please add any key to the search option. hello sir, my name is arookya mehri. can i talk to mr. subbaraj sir?'

--- Analyzing Inappropriate Words by Agent ---
❌ Inappropriate Words: The agent used potentially inappropriate words: hell

--- Analyzing Agent's Call Closing ---
❌ Agent's Closing: The agent's closing remark was not clearly identified as a good closing. Last line: 'i don't know how to pay online. i don't know how to pay online. do you know that, this is the post card of your principal? i do not know.'
   (Note: The transcript provided ends abruptly, so the actual closing might be missing or informal.)


In [6]:
!pip install ctransformers --quiet

In [7]:
!pip install -q huggingface_hub

from huggingface_hub import login

# Paste your Hugging Face token here (create it at https://huggingface.co/settings/tokens)
login(token="hf_")


In [8]:
!wget -O mistral-instruct.gguf https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

--2025-07-12 10:04:55--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 65.8.243.16, 65.8.243.92, 65.8.243.46, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/14466f9d658bf4a79f96c3f3f22759707c291cac4e62fea625e80c7d32169991?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1752318295&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MjMxODI5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2LzE0NDY2ZjlkNjU4YmY0YTc5Zjk2YzNmM2YyMjc1OTcwN

In [9]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
    "mistral-instruct.gguf",          # ✅ positional path argument
    model_type="mistral",
    max_new_tokens=512,
    context_length=2048
)


In [10]:
dialogue_output = [
    "Speaker SPEAKER_00: Please add any key to the search option. Hello Sir, My name is Arookya Mehri. Can I talk to Mr. Subbaraj Sir?",
    "Speaker SPEAKER_01: Yes, I am coming.",
    "Speaker SPEAKER_00: I have called from Maxlife Insurance. Can I talk about the policy? Are you free?",
    "Speaker SPEAKER_01: Yes, I am free.",
    "Speaker SPEAKER_00: Thank you. You have taken a policy from your name. Maxlife Fast Strike Super Plan, Policy number is 00603260. The due date is 30th September 2021. The amount to be paid is 1,20,000. The policy status is discontinued. Can I know why you are not paying?",
    "Speaker SPEAKER_01: Finance problem.",
    "Speaker SPEAKER_00: Sorry Sir.",
    "Speaker SPEAKER_01: Finance problem. I am asking for 1,20,000.",
    "Speaker SPEAKER_00: Yes Sir.",
    "Speaker SPEAKER_01: Thank you. Can I take the money from 1,20,000? Yes Sir.",
    "Speaker SPEAKER_00: So, you have a 5 year locking period. You have taken a policy in 2018. You have paid the due for 18, 19, 20 years. Can you take the amount after paying the due? How much? You can take the amount after paying the due.",
    "Speaker SPEAKER_01: How much?",
    "Speaker SPEAKER_00: Sir, you can take the amount according to your fund value. Or you can withdraw the money partially.",
    "Speaker SPEAKER_01: No, I don't have any fund value. I will withdraw it partially.",
    "Speaker SPEAKER_00: Sir, as of now, the amount is 1,87,551. But, if you pay the amount now, your fund will participate in the share market. So, there is a chance that you will get high returns.",
    "Speaker SPEAKER_01: Sir, it has been 3 years and 1,87,551. It is fair in any way.",
    "Speaker SPEAKER_00: Sir, your policy status is discontinued. Your funds are going in discontinued funds. Your fund will participate only if your policy status is active. Sir, you have not paid from 2021. So, how will your fund participate?",
    "Speaker SPEAKER_01: Sir, I have paid the loan of 1,20,000 and paid it. When will the policy end? 5 years.",
    "Speaker SPEAKER_00: You have to pay the loan for 5 years. But, the risk is for 10 years. Your fund will participate only for 10 years. If you need any emergency amount, you can withdraw the money partially. Or you can take the amount. Sir, you can take the amount until the locking period is completed.",
    "Speaker SPEAKER_01: Sir, when did the locking period start?",
    "Speaker SPEAKER_00: You have taken the amount in 2018. In this month? Yes. You have taken the amount in 30th and 31st of the 9th month.",
    "Speaker SPEAKER_01: Can I take the amount in 30th and 31st of the 9th month?",
    "Speaker SPEAKER_00: You can take the amount as soon as the payment is completed.",
    "Speaker SPEAKER_01: Can I take the amount in 20th and 31st of the 9th month?",
    "Speaker SPEAKER_00: Yes. Your locking period is completed with 2022. But, you can take the amount if you pay the loan.",
    "Speaker SPEAKER_01: Where can I get the amount if the payment is completed?",
    "Speaker SPEAKER_00: You can pay the amount in the branch office. Or you can pay the amount online. You can pay the amount online.",
    "Speaker SPEAKER_01: I don't know. You can pay the amount in the branch office. I don't know.",
    "Speaker SPEAKER_00: Sir, I will guide you. You will pay the amount online. I will help you if you get any errors.",
    "Speaker SPEAKER_01: I don't know. I don't know how to pay online. I don't know how to pay online. I don't know how to pay online. You can measure the amount in your dashboard and make the payment close. I don't know how to pay online. But, I will borrow a loan from someone.",
    "Speaker SPEAKER_00: I don't know how to pay online. I don't know how to pay online. Do you know that, this is the post card of your principal? I do not know.",
    "Speaker SPEAKER_01: He is receiving credit withains of cash. In this case, that is a credit card. It is creditor's credit. You can claim it with your credit cards. Now take the payment. Second question. First question. Yes, we can take the amount after the locking period is completed."
]

In [11]:
agent_lines = [line for line in dialogue_output if "SPEAKER_00" in line]
agent_text = "\n".join(agent_lines)

In [12]:
agent_text

"Speaker SPEAKER_00: Please add any key to the search option. Hello Sir, My name is Arookya Mehri. Can I talk to Mr. Subbaraj Sir?\nSpeaker SPEAKER_00: I have called from Maxlife Insurance. Can I talk about the policy? Are you free?\nSpeaker SPEAKER_00: Thank you. You have taken a policy from your name. Maxlife Fast Strike Super Plan, Policy number is 00603260. The due date is 30th September 2021. The amount to be paid is 1,20,000. The policy status is discontinued. Can I know why you are not paying?\nSpeaker SPEAKER_00: Sorry Sir.\nSpeaker SPEAKER_00: Yes Sir.\nSpeaker SPEAKER_00: So, you have a 5 year locking period. You have taken a policy in 2018. You have paid the due for 18, 19, 20 years. Can you take the amount after paying the due? How much? You can take the amount after paying the due.\nSpeaker SPEAKER_00: Sir, you can take the amount according to your fund value. Or you can withdraw the money partially.\nSpeaker SPEAKER_00: Sir, as of now, the amount is 1,87,551. But, if you 

In [13]:
qa_prompt = f"""
You are a conversation QA evaluator.

Analyze the following transcript from an agent and return the following:

1. Did the agent greet the customer properly?
2. Was the agent polite and professional?
3. Were there any rude, pushy, or inappropriate phrases?
4. Did the agent end the call professionally?

Return in this format:

Greeting: Yes/No - [reason]
Politeness: Yes/No - [reason]
Inappropriate Language: Yes/No - [reason]
Call Ending: Yes/No - [reason]
Agent Score: 1-5

Transcript:
{agent_text}
"""

response = llm(qa_prompt)

In [14]:
response

'Speaker SPEAKER_00: Sir, you have taken the amount in 2018. So, it will be a part of your locking period.\nSpeaker SPEAKER_00: Sir, if I withdraw the money now, I will get high returns immediately? Yes. If you withdraw now, then you will get high returns instantly. And you will get additional amount from the dividend as well.\nSpeaker SPEAKER_00: You have taken the amount in 2018, which is a part of your locking period. So, you can take the amount now and continue paying the loan for 5 years. But, if you withdraw the money now, then you will get high returns immediately? Yes. And you will get additional amount from the dividend as well.\nSpeaker SPEAKER_00: Sir, I have taken the amount in 2018, which is a part of my locking period. So, if you continue to pay the loan for 5 years and your policy status is active, then you can participate in the share market. If not, then your funds will go to discontinued funds.\nSpeaker SPEAKER_00: Sir, I have taken the amount in 2018, which is a part